# OpenAI APIs - Completions

SGLang provides OpenAI-compatible APIs to enable a smooth transition from OpenAI services to self-hosted local models.
A complete reference for the API is available in the [OpenAI API Reference](https://platform.openai.com/docs/api-reference).

This tutorial covers the following popular APIs:

- `chat/completions`
- `completions`
- `batches`

Check out other tutorials to learn about [vision APIs](https://docs.sglang.ai/backend/openai_api_vision.html) for vision-language models and [embedding APIs](https://docs.sglang.ai/backend/openai_api_embeddings.html) for embedding models.

## Launch A Server

Launch the server in your terminal and wait for it to initialize.

In [1]:
from sglang.test.test_utils import is_in_ci

if is_in_ci():
    from patch import launch_server_cmd
else:
    from sglang.utils import launch_server_cmd

from sglang.utils import wait_for_server, print_highlight, terminate_process


server_process, port = launch_server_cmd(
    "python3 -m sglang.launch_server --model-path qwen/qwen2.5-0.5b-instruct --host 0.0.0.0 --mem-fraction-static 0.8"
)

wait_for_server(f"http://localhost:{port}")
print(f"Server started on http://localhost:{port}")

[2025-06-01 01:35:38] server_args=ServerArgs(model_path='qwen/qwen2.5-0.5b-instruct', tokenizer_path='qwen/qwen2.5-0.5b-instruct', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='qwen/qwen2.5-0.5b-instruct', chat_template=None, completion_template=None, is_embedding=False, enable_multimodal=None, revision=None, host='0.0.0.0', port=38892, mem_fraction_static=0.8, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=648001713, constrained_json_whitespace_pattern=None, watchdog_timeout=300, dist_timeout=None, download_dir=None, base_gpu_id=0, gpu_id_step=1, log_level='in

[2025-06-01 01:35:50] Attention backend not set. Use fa3 backend by default.
[2025-06-01 01:35:50] Init torch distributed begin.


[2025-06-01 01:35:51] Init torch distributed ends. mem usage=0.00 GB


[2025-06-01 01:35:52] Load weight begin. avail mem=60.49 GB


[2025-06-01 01:35:52] Using model weights format ['*.safetensors']


...

[2025-06-01 01:35:53] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=32768, available_gpu_mem=59.01 GB


[2025-06-01 01:35:54] INFO:     Started server process [1566788]
[2025-06-01 01:35:54] INFO:     Waiting for application startup.
[2025-06-01 01:35:54] INFO:     Application startup complete.
[2025-06-01 01:35:54] INFO:     Uvicorn running on http://0.0.0.0:38892 (Press CTRL+C to quit)


[2025-06-01 01:35:55] INFO:     127.0.0.1:56490 - "GET /v1/models HTTP/1.1" 200 OK
[2025-06-01 01:35:55] INFO:     127.0.0.1:56504 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-06-01 01:35:55] Prefill batch. #new-seq: 1, #new-token: 6, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-06-01 01:35:57] INFO:     127.0.0.1:56508 - "POST /generate HTTP/1.1" 200 OK
[2025-06-01 01:35:57] The server is fired up and ready to roll!


Server started on http://localhost:38892


## Chat Completions

### Usage

The server fully implements the OpenAI API.
It will automatically apply the chat template specified in the Hugging Face tokenizer, if one is available.
You can also specify a custom chat template with `--chat-template` when launching the server.

In [2]:
import openai

client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="qwen/qwen2.5-0.5b-instruct",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-06-01 01:36:00] Prefill batch. #new-seq: 1, #new-token: 37, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-06-01 01:36:00] Decode batch. #running-req: 1, #token: 70, token usage: 0.00, cuda graph: False, gen throughput (token/s): 5.64, #queue-req: 0
[2025-06-01 01:36:00] INFO:     127.0.0.1:36646 - "POST /v1/chat/completions HTTP/1.1" 200 OK


### Parameters

The chat completions API accepts OpenAI Chat Completions API's parameters. Refer to [OpenAI Chat Completions API](https://platform.openai.com/docs/api-reference/chat/create) for more details.

SGLang extends the standard API with the `extra_body` parameter, allowing for additional customization. One key option within `extra_body` is `chat_template_kwargs`, which can be used to pass arguments to the chat template processor.

#### Enabling Model Thinking/Reasoning

You can use `chat_template_kwargs` to enable or disable the model's internal thinking or reasoning process output. Set `"enable_thinking": True` within `chat_template_kwargs` to include the reasoning steps in the response. This requires launching the server with a compatible reasoning parser (e.g., `--reasoning-parser qwen3` for Qwen3 models).

Here's an example demonstrating how to enable thinking and retrieve the reasoning content separately (using `separate_reasoning: True`):

```python
# Ensure the server is launched with a compatible reasoning parser, e.g.:
# python3 -m sglang.launch_server --model-path QwQ/Qwen3-32B-250415 --reasoning-parser qwen3 ...

from openai import OpenAI

# Modify OpenAI's API key and API base to use SGLang's API server.
openai_api_key = "EMPTY"
openai_api_base = f"http://127.0.0.1:{port}/v1" # Use the correct port

client = OpenAI(
    api_key=openai_api_key,
    base_url=openai_api_base,
)

model = "QwQ/Qwen3-32B-250415" # Use the model loaded by the server
messages = [{"role": "user", "content": "9.11 and 9.8, which is greater?"}]

response = client.chat.completions.create(
    model=model,
    messages=messages,
    extra_body={
        "chat_template_kwargs": {"enable_thinking": True},
        "separate_reasoning": True
    }
)

print("response.choices[0].message.reasoning_content: \n", response.choices[0].message.reasoning_content)
print("response.choices[0].message.content: \n", response.choices[0].message.content)
```

**Example Output:**

```
response.choices[0].message.reasoning_content: 
 Okay, so I need to figure out which number is greater between 9.11 and 9.8. Hmm, let me think. Both numbers start with 9, right? So the whole number part is the same. That means I need to look at the decimal parts to determine which one is bigger.
...
Therefore, after checking multiple methods—aligning decimals, subtracting, converting to fractions, and using a real-world analogy—it's clear that 9.8 is greater than 9.11.

response.choices[0].message.content: 
 To determine which number is greater between **9.11** and **9.8**, follow these steps:
...
**Answer**:  
9.8 is greater than 9.11.
```

Setting `"enable_thinking": False` (or omitting it) will result in `reasoning_content` being `None`.

Here is an example of a detailed chat completion request using standard OpenAI parameters:

In [3]:
response = client.chat.completions.create(
    model="qwen/qwen2.5-0.5b-instruct",
    messages=[
        {
            "role": "system",
            "content": "You are a knowledgeable historian who provides concise responses.",
        },
        {"role": "user", "content": "Tell me about ancient Rome"},
        {
            "role": "assistant",
            "content": "Ancient Rome was a civilization centered in Italy.",
        },
        {"role": "user", "content": "What were their major achievements?"},
    ],
    temperature=0.3,  # Lower temperature for more focused responses
    max_tokens=128,  # Reasonable length for a concise response
    top_p=0.95,  # Slightly higher for better fluency
    presence_penalty=0.2,  # Mild penalty to avoid repetition
    frequency_penalty=0.2,  # Mild penalty for more natural language
    n=1,  # Single response is usually more stable
    seed=42,  # Keep for reproducibility
)

print_highlight(response.choices[0].message.content)

[2025-06-01 01:36:00] Prefill batch. #new-seq: 1, #new-token: 49, #cached-token: 5, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-06-01 01:36:01] Decode batch. #running-req: 1, #token: 88, token usage: 0.00, cuda graph: False, gen throughput (token/s): 115.74, #queue-req: 0


[2025-06-01 01:36:01] Decode batch. #running-req: 1, #token: 128, token usage: 0.01, cuda graph: False, gen throughput (token/s): 135.62, #queue-req: 0


[2025-06-01 01:36:01] Decode batch. #running-req: 1, #token: 168, token usage: 0.01, cuda graph: False, gen throughput (token/s): 129.68, #queue-req: 0
[2025-06-01 01:36:01] INFO:     127.0.0.1:36646 - "POST /v1/chat/completions HTTP/1.1" 200 OK


Streaming mode is also supported.

In [4]:
stream = client.chat.completions.create(
    model="qwen/qwen2.5-0.5b-instruct",
    messages=[{"role": "user", "content": "Say this is a test"}],
    stream=True,
)
for chunk in stream:
    if chunk.choices[0].delta.content is not None:
        print(chunk.choices[0].delta.content, end="")

[2025-06-01 01:36:01] INFO:     127.0.0.1:36646 - "POST /v1/chat/completions HTTP/1.1" 200 OK
[2025-06-01 01:36:01] Prefill batch. #new-seq: 1, #new-token: 10, #cached-token: 24, token usage: 0.00, #running-req: 0, #queue-req: 0
Yes, I'm here to assist you. If

 you need help with anything or if there's something I can help you with

[2025-06-01 01:36:02] Decode batch. #running-req: 1, #token: 60, token usage: 0.00, cuda graph: False, gen throughput (token/s): 127.19, #queue-req: 0
, feel free to ask!

## Completions

### Usage
Completions API is similar to Chat Completions API, but without the `messages` parameter or chat templates.

In [5]:
response = client.completions.create(
    model="qwen/qwen2.5-0.5b-instruct",
    prompt="List 3 countries and their capitals.",
    temperature=0,
    max_tokens=64,
    n=1,
    stop=None,
)

print_highlight(f"Response: {response}")

[2025-06-01 01:36:02] Prefill batch. #new-seq: 1, #new-token: 8, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-06-01 01:36:02] Decode batch. #running-req: 1, #token: 42, token usage: 0.00, cuda graph: False, gen throughput (token/s): 117.66, #queue-req: 0


[2025-06-01 01:36:02] INFO:     127.0.0.1:36646 - "POST /v1/completions HTTP/1.1" 200 OK


### Parameters

The completions API accepts OpenAI Completions API's parameters.  Refer to [OpenAI Completions API](https://platform.openai.com/docs/api-reference/completions/create) for more details.

Here is an example of a detailed completions request:

In [6]:
response = client.completions.create(
    model="qwen/qwen2.5-0.5b-instruct",
    prompt="Write a short story about a space explorer.",
    temperature=0.7,  # Moderate temperature for creative writing
    max_tokens=150,  # Longer response for a story
    top_p=0.9,  # Balanced diversity in word choice
    stop=["\n\n", "THE END"],  # Multiple stop sequences
    presence_penalty=0.3,  # Encourage novel elements
    frequency_penalty=0.3,  # Reduce repetitive phrases
    n=1,  # Generate one completion
    seed=123,  # For reproducible results
)

print_highlight(f"Response: {response}")

[2025-06-01 01:36:02] Prefill batch. #new-seq: 1, #new-token: 9, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-06-01 01:36:02] Decode batch. #running-req: 1, #token: 19, token usage: 0.00, cuda graph: False, gen throughput (token/s): 67.19, #queue-req: 0


[2025-06-01 01:36:03] Decode batch. #running-req: 1, #token: 59, token usage: 0.00, cuda graph: False, gen throughput (token/s): 124.57, #queue-req: 0


[2025-06-01 01:36:03] Decode batch. #running-req: 1, #token: 99, token usage: 0.00, cuda graph: False, gen throughput (token/s): 135.41, #queue-req: 0


[2025-06-01 01:36:03] Decode batch. #running-req: 1, #token: 139, token usage: 0.01, cuda graph: False, gen throughput (token/s): 126.59, #queue-req: 0
[2025-06-01 01:36:03] INFO:     127.0.0.1:36646 - "POST /v1/completions HTTP/1.1" 200 OK


## Structured Outputs (JSON, Regex, EBNF)

For OpenAI compatible structured outputs API, refer to [Structured Outputs](https://docs.sglang.ai/backend/structured_outputs.html#OpenAI-Compatible-API) for more details.


## Batches

Batches API for chat completions and completions are also supported. You can upload your requests in `jsonl` files, create a batch job, and retrieve the results when the batch job is completed (which takes longer but costs less).

The batches APIs are:

- `batches`
- `batches/{batch_id}/cancel`
- `batches/{batch_id}`

Here is an example of a batch job for chat completions, completions are similar.


In [7]:
import json
import time
from openai import OpenAI

client = OpenAI(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

requests = [
    {
        "custom_id": "request-1",
        "method": "POST",
        "url": "/chat/completions",
        "body": {
            "model": "qwen/qwen2.5-0.5b-instruct",
            "messages": [
                {"role": "user", "content": "Tell me a joke about programming"}
            ],
            "max_tokens": 50,
        },
    },
    {
        "custom_id": "request-2",
        "method": "POST",
        "url": "/chat/completions",
        "body": {
            "model": "qwen/qwen2.5-0.5b-instruct",
            "messages": [{"role": "user", "content": "What is Python?"}],
            "max_tokens": 50,
        },
    },
]

input_file_path = "batch_requests.jsonl"

with open(input_file_path, "w") as f:
    for req in requests:
        f.write(json.dumps(req) + "\n")

with open(input_file_path, "rb") as f:
    file_response = client.files.create(file=f, purpose="batch")

batch_response = client.batches.create(
    input_file_id=file_response.id,
    endpoint="/v1/chat/completions",
    completion_window="24h",
)

print_highlight(f"Batch job created with ID: {batch_response.id}")

[2025-06-01 01:36:04] INFO:     127.0.0.1:36652 - "POST /v1/files HTTP/1.1" 200 OK


[2025-06-01 01:36:04] INFO:     127.0.0.1:36652 - "POST /v1/batches HTTP/1.1" 200 OK


[2025-06-01 01:36:04] Prefill batch. #new-seq: 2, #new-token: 20, #cached-token: 48, token usage: 0.00, #running-req: 0, #queue-req: 0


In [8]:
while batch_response.status not in ["completed", "failed", "cancelled"]:
    time.sleep(3)
    print(f"Batch job status: {batch_response.status}...trying again in 3 seconds...")
    batch_response = client.batches.retrieve(batch_response.id)

if batch_response.status == "completed":
    print("Batch job completed successfully!")
    print(f"Request counts: {batch_response.request_counts}")

    result_file_id = batch_response.output_file_id
    file_response = client.files.content(result_file_id)
    result_content = file_response.read().decode("utf-8")

    results = [
        json.loads(line) for line in result_content.split("\n") if line.strip() != ""
    ]

    for result in results:
        print_highlight(f"Request {result['custom_id']}:")
        print_highlight(f"Response: {result['response']}")

    print_highlight("Cleaning up files...")
    # Only delete the result file ID since file_response is just content
    client.files.delete(result_file_id)
else:
    print_highlight(f"Batch job failed with status: {batch_response.status}")
    if hasattr(batch_response, "errors"):
        print_highlight(f"Errors: {batch_response.errors}")

[2025-06-01 01:36:04] Decode batch. #running-req: 2, #token: 94, token usage: 0.00, cuda graph: False, gen throughput (token/s): 117.77, #queue-req: 0


Batch job status: validating...trying again in 3 seconds...
[2025-06-01 01:36:07] INFO:     127.0.0.1:36652 - "GET /v1/batches/batch_117c26b2-e854-422e-a61c-ef602ee38f96 HTTP/1.1" 200 OK
Batch job completed successfully!
Request counts: BatchRequestCounts(completed=2, failed=0, total=2)
[2025-06-01 01:36:07] INFO:     127.0.0.1:36652 - "GET /v1/files/backend_result_file-27579117-8813-4f0c-8573-38047a916531/content HTTP/1.1" 200 OK


[2025-06-01 01:36:07] INFO:     127.0.0.1:36652 - "DELETE /v1/files/backend_result_file-27579117-8813-4f0c-8573-38047a916531 HTTP/1.1" 200 OK


It takes a while to complete the batch job. You can use these two APIs to retrieve the batch job status or cancel the batch job.

1. `batches/{batch_id}`: Retrieve the batch job status.
2. `batches/{batch_id}/cancel`: Cancel the batch job.

Here is an example to check the batch job status.

In [9]:
import json
import time
from openai import OpenAI

client = OpenAI(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

requests = []
for i in range(20):
    requests.append(
        {
            "custom_id": f"request-{i}",
            "method": "POST",
            "url": "/chat/completions",
            "body": {
                "model": "qwen/qwen2.5-0.5b-instruct",
                "messages": [
                    {
                        "role": "system",
                        "content": f"{i}: You are a helpful AI assistant",
                    },
                    {
                        "role": "user",
                        "content": "Write a detailed story about topic. Make it very long.",
                    },
                ],
                "max_tokens": 64,
            },
        }
    )

input_file_path = "batch_requests.jsonl"
with open(input_file_path, "w") as f:
    for req in requests:
        f.write(json.dumps(req) + "\n")

with open(input_file_path, "rb") as f:
    uploaded_file = client.files.create(file=f, purpose="batch")

batch_job = client.batches.create(
    input_file_id=uploaded_file.id,
    endpoint="/v1/chat/completions",
    completion_window="24h",
)

print_highlight(f"Created batch job with ID: {batch_job.id}")
print_highlight(f"Initial status: {batch_job.status}")

time.sleep(10)

max_checks = 5
for i in range(max_checks):
    batch_details = client.batches.retrieve(batch_id=batch_job.id)

    print_highlight(
        f"Batch job details (check {i+1} / {max_checks}) // ID: {batch_details.id} // Status: {batch_details.status} // Created at: {batch_details.created_at} // Input file ID: {batch_details.input_file_id} // Output file ID: {batch_details.output_file_id}"
    )
    print_highlight(
        f"<strong>Request counts: Total: {batch_details.request_counts.total} // Completed: {batch_details.request_counts.completed} // Failed: {batch_details.request_counts.failed}</strong>"
    )

    time.sleep(3)

[2025-06-01 01:36:07] INFO:     127.0.0.1:55846 - "POST /v1/files HTTP/1.1" 200 OK
[2025-06-01 01:36:07] INFO:     127.0.0.1:55846 - "POST /v1/batches HTTP/1.1" 200 OK


[2025-06-01 01:36:07] Prefill batch. #new-seq: 20, #new-token: 610, #cached-token: 60, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-06-01 01:36:07] Decode batch. #running-req: 20, #token: 903, token usage: 0.04, cuda graph: False, gen throughput (token/s): 105.83, #queue-req: 0


[2025-06-01 01:36:07] Decode batch. #running-req: 19, #token: 1618, token usage: 0.08, cuda graph: False, gen throughput (token/s): 2357.12, #queue-req: 0


[2025-06-01 01:36:17] INFO:     127.0.0.1:47144 - "GET /v1/batches/batch_febaab67-a578-4c16-bdf6-99a7c8590615 HTTP/1.1" 200 OK


[2025-06-01 01:36:20] INFO:     127.0.0.1:47144 - "GET /v1/batches/batch_febaab67-a578-4c16-bdf6-99a7c8590615 HTTP/1.1" 200 OK


[2025-06-01 01:36:23] INFO:     127.0.0.1:47144 - "GET /v1/batches/batch_febaab67-a578-4c16-bdf6-99a7c8590615 HTTP/1.1" 200 OK


[2025-06-01 01:36:26] INFO:     127.0.0.1:47144 - "GET /v1/batches/batch_febaab67-a578-4c16-bdf6-99a7c8590615 HTTP/1.1" 200 OK


[2025-06-01 01:36:29] INFO:     127.0.0.1:47144 - "GET /v1/batches/batch_febaab67-a578-4c16-bdf6-99a7c8590615 HTTP/1.1" 200 OK


Here is an example to cancel a batch job.

In [10]:
import json
import time
from openai import OpenAI
import os

client = OpenAI(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

requests = []
for i in range(5000):
    requests.append(
        {
            "custom_id": f"request-{i}",
            "method": "POST",
            "url": "/chat/completions",
            "body": {
                "model": "qwen/qwen2.5-0.5b-instruct",
                "messages": [
                    {
                        "role": "system",
                        "content": f"{i}: You are a helpful AI assistant",
                    },
                    {
                        "role": "user",
                        "content": "Write a detailed story about topic. Make it very long.",
                    },
                ],
                "max_tokens": 128,
            },
        }
    )

input_file_path = "batch_requests.jsonl"
with open(input_file_path, "w") as f:
    for req in requests:
        f.write(json.dumps(req) + "\n")

with open(input_file_path, "rb") as f:
    uploaded_file = client.files.create(file=f, purpose="batch")

batch_job = client.batches.create(
    input_file_id=uploaded_file.id,
    endpoint="/v1/chat/completions",
    completion_window="24h",
)

print_highlight(f"Created batch job with ID: {batch_job.id}")
print_highlight(f"Initial status: {batch_job.status}")

time.sleep(10)

try:
    cancelled_job = client.batches.cancel(batch_id=batch_job.id)
    print_highlight(f"Cancellation initiated. Status: {cancelled_job.status}")
    assert cancelled_job.status == "cancelling"

    # Monitor the cancellation process
    while cancelled_job.status not in ["failed", "cancelled"]:
        time.sleep(3)
        cancelled_job = client.batches.retrieve(batch_job.id)
        print_highlight(f"Current status: {cancelled_job.status}")

    # Verify final status
    assert cancelled_job.status == "cancelled"
    print_highlight("Batch job successfully cancelled")

except Exception as e:
    print_highlight(f"Error during cancellation: {e}")
    raise e

finally:
    try:
        del_response = client.files.delete(uploaded_file.id)
        if del_response.deleted:
            print_highlight("Successfully cleaned up input file")
        if os.path.exists(input_file_path):
            os.remove(input_file_path)
            print_highlight("Successfully deleted local batch_requests.jsonl file")
    except Exception as e:
        print_highlight(f"Error cleaning up: {e}")
        raise e

[2025-06-01 01:36:32] INFO:     127.0.0.1:58316 - "POST /v1/files HTTP/1.1" 200 OK
[2025-06-01 01:36:32] INFO:     127.0.0.1:58316 - "POST /v1/batches HTTP/1.1" 200 OK


[2025-06-01 01:36:33] Prefill batch. #new-seq: 129, #new-token: 3290, #cached-token: 1115, token usage: 0.03, #running-req: 0, #queue-req: 131
[2025-06-01 01:36:33] Prefill batch. #new-seq: 31, #new-token: 930, #cached-token: 155, token usage: 0.19, #running-req: 129, #queue-req: 923


[2025-06-01 01:36:33] Prefill batch. #new-seq: 8, #new-token: 240, #cached-token: 40, token usage: 0.33, #running-req: 157, #queue-req: 4832
[2025-06-01 01:36:34] Decode batch. #running-req: 165, #token: 9965, token usage: 0.49, cuda graph: False, gen throughput (token/s): 192.65, #queue-req: 4832


[2025-06-01 01:36:34] Decode batch. #running-req: 160, #token: 16061, token usage: 0.78, cuda graph: False, gen throughput (token/s): 16599.27, #queue-req: 4832


[2025-06-01 01:36:34] KV cache pool is full. Retract requests. #retracted_reqs: 22, #new_token_ratio: 0.6017 -> 0.9219
[2025-06-01 01:36:34] Decode batch. #running-req: 138, #token: 19463, token usage: 0.95, cuda graph: False, gen throughput (token/s): 15475.76, #queue-req: 4854
[2025-06-01 01:36:34] KV cache pool is full. Retract requests. #retracted_reqs: 17, #new_token_ratio: 0.9028 -> 1.0000
[2025-06-01 01:36:34] Prefill batch. #new-seq: 8, #new-token: 240, #cached-token: 40, token usage: 0.88, #running-req: 121, #queue-req: 4863


...

[2025-06-01 01:36:41] Decode batch. #running-req: 142, #token: 18382, token usage: 0.90, cuda graph: False, gen throughput (token/s): 13761.57, #queue-req: 4165
[2025-06-01 01:36:41] KV cache pool is full. Retract requests. #retracted_reqs: 19, #new_token_ratio: 0.7341 -> 1.0000
[2025-06-01 01:36:41] Prefill batch. #new-seq: 6, #new-token: 190, #cached-token: 20, token usage: 0.89, #running-req: 123, #queue-req: 4178


[2025-06-01 01:36:41] Prefill batch. #new-seq: 88, #new-token: 2722, #cached-token: 358, token usage: 0.26, #running-req: 41, #queue-req: 4090
[2025-06-01 01:36:42] Prefill batch. #new-seq: 39, #new-token: 1263, #cached-token: 133, token usage: 0.19, #running-req: 91, #queue-req: 4051
[2025-06-01 01:36:42] Decode batch. #running-req: 130, #token: 5673, token usage: 0.28, cuda graph: False, gen throughput (token/s): 11872.52, #queue-req: 4051


[2025-06-01 01:36:42] Prefill batch. #new-seq: 3, #new-token: 90, #cached-token: 18, token usage: 0.38, #running-req: 129, #queue-req: 4048
[2025-06-01 01:36:42] Prefill batch. #new-seq: 2, #new-token: 60, #cached-token: 12, token usage: 0.40, #running-req: 131, #queue-req: 4046
[2025-06-01 01:36:42] Prefill batch. #new-seq: 1, #new-token: 30, #cached-token: 6, token usage: 0.41, #running-req: 132, #queue-req: 4045
[2025-06-01 01:36:42] Prefill batch. #new-seq: 1, #new-token: 30, #cached-token: 6, token usage: 0.44, #running-req: 132, #queue-req: 4044
[2025-06-01 01:36:42] INFO:     127.0.0.1:47442 - "POST /v1/batches/batch_1d196ff5-4b10-4b58-a3e3-88414c1621ba/cancel HTTP/1.1" 200 OK


[2025-06-01 01:36:45] INFO:     127.0.0.1:47442 - "GET /v1/batches/batch_1d196ff5-4b10-4b58-a3e3-88414c1621ba HTTP/1.1" 200 OK


[2025-06-01 01:36:45] INFO:     127.0.0.1:47442 - "DELETE /v1/files/backend_input_file-4d6bdcd6-9029-4a59-ba59-5f11410844b3 HTTP/1.1" 200 OK


In [11]:
terminate_process(server_process)

[2025-06-01 01:36:45] Child process unexpectedly failed with exitcode=9. pid=1567136
[2025-06-01 01:36:45] Child process unexpectedly failed with exitcode=9. pid=1567064
